# Set up

In [1]:
import pandas as pd
import pickle
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")
import matplotlib.ticker as ticker
import seaborn as sns
import matplotlib.dates as dates
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf

In [2]:
dir_ = 'C:/Users/Palida Yingwatchara/DSE/m5-forecasting-accuracy/'

In [3]:
calen_df = pd.read_csv(dir_ + 'calendar.csv' )
sales_train_eval_df = pd.read_csv(dir_+'sales_train_evaluation.csv')
prices_df = pd.read_csv(dir_+'sell_prices.csv')
sales_train_valid_df =  pd.read_csv(dir_+'sales_train_validation.csv')
sample_submission_df = pd.read_csv(dir_+'sample_submission.csv')

In [4]:
calen_df.head(3)


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0


In [5]:
calen_df = calen_df.fillna("no_event")

In [6]:
calen_df.drop("snap_CA",  axis='columns', inplace=True)
calen_df.drop("snap_TX",  axis='columns', inplace=True)
calen_df.drop("snap_WI",  axis='columns', inplace=True)
calen_df.drop("wday",  axis='columns', inplace=True)

In [7]:
#Replace Month
month=['Jan','Feb','March','April','May','June','July','Aug','Sept','Oct','Nov','Dec']
for i in range(len(month)):
  calen_df['month']=calen_df['month'].replace(i+1,month[i])

In [8]:
#isweekend feature
def isweekend(s) :
    if s == "Saturday" or s == "Sunday" :
        return "weekend"
    return "weekday"
calen_df['is_weekend'] = calen_df['weekday'].apply(isweekend)


In [9]:
calen_df[calen_df.event_name_1 != "no_event"]

,date,wm_yr_wk,weekday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,is_weekend
8,2011-02-06,11102,Sunday,Feb,2011,d_9,SuperBowl,Sporting,no_event,no_event,weekend
16,2011-02-14,11103,Monday,Feb,2011,d_17,ValentinesDay,Cultural,no_event,no_event,weekday
23,2011-02-21,11104,Monday,Feb,2011,d_24,PresidentsDay,National,no_event,no_event,weekday
39,2011-03-09,11106,Wednesday,March,2011,d_40,LentStart,Religious,no_event,no_event,weekday
46,2011-03-16,11107,Wednesday,March,2011,d_47,LentWeek2,Religious,no_event,no_event,weekday
...,...,...,...,...,...,...,...,...,...,...,...
1926,2016-05-08,11615,Sunday,May,2016,d_1927,Mother's day,Cultural,no_event,no_event,weekend
1948,2016-05-30,11618,Monday,May,2016,d_1949,MemorialDay,National,no_event,no_event,weekday
1951,2016-06-02,11618,Thursday,June,2016,d_1952,NBAFinalsStart,Sporting,no_event,no_event,weekday
1956,2016-06-07,11619,Tuesday,June,2016,d_1957,Ramadan starts,Religious,no_event,no_event,weekday


In [10]:
calen_df.date = pd.to_datetime(calen_df.date)

In [11]:
#calen_df["date"].dt.day

In [12]:
## Memory Reducer
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
calen_df = reduce_mem_usage(calen_df)

Mem. usage decreased to  0.14 Mb (13.6% reduction)


In [14]:
sales_train_eval_df = reduce_mem_usage(sales_train_eval_df)

Mem. usage decreased to 96.13 Mb (78.8% reduction)


In [15]:
prices_df =  reduce_mem_usage(prices_df)

Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [16]:
prices_df.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [17]:
sales_train_eval_df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [18]:
melted_df =pd.melt(sales_train_eval_df, id_vars=['id','item_id','dept_id','cat_id','store_id','state_id'],var_name='d',value_name='sale_units')

In [19]:
melted_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale_units
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0
...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0


In [20]:
melted_df =pd.merge(melted_df,calen_df,on='d',how='left')

In [21]:
calen_df

,date,wm_yr_wk,weekday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,is_weekend
0,2011-01-29,11101,Saturday,Jan,2011,d_1,no_event,no_event,no_event,no_event,weekend
1,2011-01-30,11101,Sunday,Jan,2011,d_2,no_event,no_event,no_event,no_event,weekend
2,2011-01-31,11101,Monday,Jan,2011,d_3,no_event,no_event,no_event,no_event,weekday
3,2011-02-01,11101,Tuesday,Feb,2011,d_4,no_event,no_event,no_event,no_event,weekday
4,2011-02-02,11101,Wednesday,Feb,2011,d_5,no_event,no_event,no_event,no_event,weekday
...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,June,2016,d_1965,no_event,no_event,no_event,no_event,weekday
1965,2016-06-16,11620,Thursday,June,2016,d_1966,no_event,no_event,no_event,no_event,weekday
1966,2016-06-17,11620,Friday,June,2016,d_1967,no_event,no_event,no_event,no_event,weekday
1967,2016-06-18,11621,Saturday,June,2016,d_1968,no_event,no_event,no_event,no_event,weekend


In [22]:
melted_df

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale_units,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,is_weekend
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59181085,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend
59181086,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend
59181087,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend
59181088,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend


In [23]:
revenue=pd.merge(melted_df,prices_df,on=['item_id','store_id','wm_yr_wk'],how='left')

In [24]:
revenue["cost"] = revenue.sale_units * revenue.sell_price 

In [25]:
revenue.dropna(inplace = True)

In [26]:
revenue = revenue.reset_index()
revenue.drop("index",  axis='columns', inplace=True)

In [27]:
revenue

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale_units,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,is_weekend,sell_price,cost
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.459961,5.519531
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,1.559570,3.119141
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,3.169922,0.000000
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,5.980469,0.000000
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.700195,2.800781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.980469,2.980469
46881673,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.480469,0.000000
46881674,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,3.980469,7.960938
46881675,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,1.280273,0.000000


In [28]:
revenue = reduce_mem_usage(revenue)

Mem. usage decreased to 5812.28 Mb (1.5% reduction)


In [29]:
##Sale per day

df1 = melted_df.groupby('weekday').sum()["sale_units"].reset_index()
df1['percent']=np.round(df1['sale_units']/df1['sale_units'].sum()*100,2)
df1=df1.sort_values('percent')
df1

,weekday,sale_units,percent
6,Wednesday,8346126.0,12.47
4,Thursday,8398928.0,12.55
5,Tuesday,8448634.0,12.62
1,Monday,9139596.0,13.66
0,Friday,9517020.0,14.22
3,Sunday,11482526.0,17.16
2,Saturday,11594343.0,17.32


In [151]:
fig = px.bar(df1, x='weekday', y='percent',text='percent', title="Sales per day")
fig.show() 

In [45]:
## Sales per month
melted_df["month_index"] = melted_df["date"].dt.month
df2 = melted_df.groupby(["month","month_index"]).sum()["sale_units"].reset_index()
df2['percent']=np.round(df2['sale_units']/df2['sale_units'].sum()*100,2)
df2=df2.sort_values('month_index')

In [51]:
fig2 = px.line(df2, x='month', y='percent',text='percent', title="Sales per month")
fig2.update_traces(textposition="bottom right")
fig2.show() 

In [52]:
##Sales per year
df3=melted_df.groupby('year').sum()["sale_units"].reset_index()
df3['percent']=np.round(df3['sale_units']/df3['sale_units'].sum()*100,2)
df3=df3.sort_values('year')
df3

,year,sale_units,percent
0,2011,8856585.0,13.23
1,2012,12061837.0,18.02
2,2013,13135753.0,19.63
3,2014,13089776.0,19.56
4,2015,13800811.0,20.62
5,2016,5982411.0,8.94


In [53]:
fig3 = px.line(df3, x='year', y='percent',text='percent', title="Sales per year")
fig3.update_traces(textposition="bottom right")
fig3.show() 

In [56]:
# Average Sales per events
df4=melted_df.groupby('event_type_1').mean()["sale_units"].reset_index()
df4['percent']=np.round(df4['sale_units']/df4['sale_units'].sum()*100,2)
df4

,event_type_1,sale_units,percent
0,Cultural,1.138688,20.54
1,National,0.966170,17.43
2,Religious,1.129099,20.37
3,Sporting,1.174026,21.18
4,no_event,1.135105,20.48


In [57]:
fig4 = px.bar(df4, x='event_type_1', y='percent',text='percent', title="Average Sales per event",   labels={
                     "event_type_1": "Event types",
                     "percent": "Average Sales (percent)"  })         
fig4.show() 

In [79]:
df5=melted_df.groupby(['event_type_1','event_name_1']).sum()["sale_units"].reset_index()
df5

,event_type_1,event_name_1,sale_units
0,Cultural,Cinco De Mayo,172057.0
1,Cultural,Easter,197588.0
2,Cultural,Father's day,146248.0
3,Cultural,Halloween,150297.0
4,Cultural,Mother's day,216666.0
5,Cultural,StPatricksDay,200758.0
6,Cultural,ValentinesDay,200974.0
7,National,Christmas,78.0
8,National,ColumbusDay,175824.0
9,National,IndependenceDay,168554.0


In [80]:
df5.drop(30,inplace=True)
df5.sort_values("sale_units")

,event_type_1,event_name_1,sale_units
7,National,Christmas,78.0
15,National,Thanksgiving,105650.0
13,National,NewYear,128842.0
2,Cultural,Father's day,146248.0
3,Cultural,Halloween,150297.0
28,Sporting,NBAFinalsStart,159372.0
12,National,MemorialDay,163498.0
22,Religious,OrthodoxChristmas,166721.0
11,National,MartinLutherKingDay,166893.0
17,Religious,Chanukah End,167134.0


In [84]:
df5= df5.sort_values("sale_units", ascending=False)
df5 =df5[:10]
df5

,event_type_1,event_name_1,sale_units
29,Sporting,SuperBowl,245544.0
24,Religious,Pesach End,217680.0
4,Cultural,Mother's day,216666.0
25,Religious,Purim End,215742.0
10,National,LaborDay,210773.0
14,National,PresidentsDay,206678.0
23,Religious,OrthodoxEaster,206140.0
6,Cultural,ValentinesDay,200974.0
5,Cultural,StPatricksDay,200758.0
1,Cultural,Easter,197588.0


In [87]:
df6 = melted_df.groupby(['dept_id',"year"]).sum()["sale_units"].reset_index()
#https://stackoverflow.com/questions/60595374/typeerror-cannot-insert-an-item-into-a-categoricalindex-that-is-not-already-an

In [88]:
df6 

,dept_id,year,sale_units
0,FOODS_1,2011,610070.0
1,FOODS_1,2012,1011192.0
2,FOODS_1,2013,1038468.0
3,FOODS_1,2014,984877.0
4,FOODS_1,2015,1087217.0
5,FOODS_1,2016,458576.0
6,FOODS_2,2011,1139010.0
7,FOODS_2,2012,1318851.0
8,FOODS_2,2013,1375190.0
9,FOODS_2,2014,1437226.0


In [90]:
fig6 = px.line(df6, x='year', y='sale_units',color='dept_id', title="Sales per department", labels={
                     "sale_units": "Total Sales (units)"
                      })

fig6.show() 

In [92]:
##Sale per category
df7 = melted_df.groupby(["cat_id","year"]).sum()["sale_units"].reset_index()
df7

,cat_id,year,sale_units
0,FOODS,2011,6258864.0
1,FOODS,2012,8660268.0
2,FOODS,2013,8991482.0
3,FOODS,2014,8995525.0
4,FOODS,2015,9013855.0
5,FOODS,2016,4002433.0
6,HOBBIES,2011,939151.0
7,HOBBIES,2012,959837.0
8,HOBBIES,2013,1208059.0
9,HOBBIES,2014,1183846.0


In [93]:
fig7 = px.line(df7, x='year', y='sale_units',color='cat_id', title="Sales per category", labels={
                     "sale_units": "Total Sales (units)"
                      })

fig7.show() 

In [99]:
revenue

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale_units,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,is_weekend,sell_price,cost
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.459961,5.519531
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,1.559570,3.119141
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,3.169922,0.000000
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,5.980469,0.000000
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.700195,2.800781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.980469,2.980469
46881673,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.480469,0.000000
46881674,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,3.980469,7.960938
46881675,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,1.280273,0.000000


In [95]:
revenue.dtypes

id                      object
item_id                 object
dept_id                 object
cat_id                  object
store_id                object
state_id                object
d                       object
sale_units               int16
date            datetime64[ns]
wm_yr_wk                 int16
weekday                 object
month                   object
year                     int16
event_name_1            object
event_type_1            object
event_name_2            object
event_type_2            object
is_weekend              object
sell_price             float16
cost                   float16
dtype: object

In [100]:
revenue['cost'] = revenue['cost'].astype(np.float16)

In [103]:
df8 = revenue.groupby(["cat_id"]).sum()["cost"].reset_index()
df8

,cat_id,cost
0,FOODS,inf
1,HOBBIES,inf
2,HOUSEHOLD,inf


In [104]:
cat=['FOODS','HOBBIES','HOUSEHOLD']
l=[]
for i in cat:
  s=[]
  s=revenue[revenue['cat_id']==i]['cost'].tolist()
  print(s)
  l.append(np.round(sum(s),2))
d=pd.DataFrame(columns=['cat','cost'])
d['cat']=cat
d['cost']=l

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [105]:
d

,cat,cost
0,FOODS,1.111424e+08
1,HOBBIES,2.332122e+07
2,HOUSEHOLD,5.711502e+07


In [106]:
d["percent"] = d["cost"]/d["cost"].sum()*100
d

,cat,cost,percent
0,FOODS,1.111424e+08,58.013986
1,HOBBIES,2.332122e+07,12.173182
2,HOUSEHOLD,5.711502e+07,29.812832


In [109]:
fig8 = px.pie(d, values='cost', names='cat', title='Percentage of Revenue per Category')
fig8.show()

In [111]:
revenue

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale_units,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,is_weekend,sell_price,cost
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.459961,5.519531
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,1.559570,3.119141
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,3.169922,0.000000
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,5.980469,0.000000
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.700195,2.800781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.980469,2.980469
46881673,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.480469,0.000000
46881674,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,3.980469,7.960938
46881675,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,1.280273,0.000000


In [113]:
df9 = revenue.groupby(["store_id"]).sum()["sale_units"].reset_index()
df9 

,store_id,sale_units
0,CA_1,7832248.0
1,CA_2,5818395.0
2,CA_3,11363540.0
3,CA_4,4182534.0
4,TX_1,5692823.0
5,TX_2,7329642.0
6,TX_3,6205940.0
7,WI_1,5261506.0
8,WI_2,6697988.0
9,WI_3,6542557.0


In [ ]:
fig9 = px.pie(d, values='cost', names='cat', title='Percentage of Revenue per Category')
fig9.show()

In [115]:
df10 = revenue.groupby(["state_id"]).sum()["sale_units"].reset_index()
df10

,state_id,sale_units
0,CA,29196717.0
1,TX,19228405.0
2,WI,18502051.0


In [116]:
cat=['FOODS','HOBBIES','HOUSEHOLD']
l=[]
for i in cat:
  s=[]
  s=revenue[revenue['cat_id']==i]['cost'].tolist()
  print(s)
  l.append(np.round(sum(s),2))
d=pd.DataFrame(columns=['cat','cost'])
d['cat']=cat
d['cost']=l

,store_id,cost
0,CA_1,inf
1,CA_2,inf
2,CA_3,inf
3,CA_4,inf
4,TX_1,inf
5,TX_2,inf
6,TX_3,inf
7,WI_1,inf
8,WI_2,inf
9,WI_3,inf


In [156]:
store = ['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3']
l=[]
for i in store:
  s=[]
  s=revenue[revenue['store_id']==i]['cost'].tolist() #list of the cost of each store
  l.append(np.round(sum(s),2))
d1=pd.DataFrame(columns=['store_id','cost'])
d1['store_id']=store
d1['cost']=l

In [127]:
state = ['CA','TX','WI']
l=[]
for i in state:
  s=[]
  s=revenue[revenue['state_id']==i]['cost'].tolist() #list of the cost of each state
  l.append(np.round(sum(s),2))
d2=pd.DataFrame(columns=['state_id','cost'])
d2['state_id']=state
d2['cost']=l

In [157]:
fig9 = px.bar(d1, x='store_id', y='cost',text='cost', title="Revnue per store",   labels={
                     "store_id": "store",
                     "cost": "Revenue"  })         
fig9.show() 

In [152]:
d1

,store_id,cost
0,CA_1,22954413.86
1,CA_2,17848070.61
2,CA_3,32699356.07
3,CA_4,12465717.07
4,TX_1,16037402.89
5,TX_2,20893139.89
6,TX_3,18190481.20
7,WI_1,15107464.04
8,WI_2,18132223.04
9,WI_3,17250373.71


In [153]:
d2

,state_id,cost
0,CA,85967557.62
1,TX,55121023.99
2,WI,50490060.80


In [136]:
revenue

,id,item_id,dept_id,cat_id,store_id,state_id,d,sale_units,date,wm_yr_wk,weekday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,is_weekend,sell_price,cost
0,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_008,HOBBIES_1,HOBBIES,CA_1,CA,d_1,12,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.459961,5.519531
1,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_009,HOBBIES_1,HOBBIES,CA_1,CA,d_1,2,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,1.559570,3.119141
2,HOBBIES_1_010_CA_1_evaluation,HOBBIES_1_010,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,3.169922,0.000000
3,HOBBIES_1_012_CA_1_evaluation,HOBBIES_1_012,HOBBIES_1,HOBBIES,CA_1,CA,d_1,0,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,5.980469,0.000000
4,HOBBIES_1_015_CA_1_evaluation,HOBBIES_1_015,HOBBIES_1,HOBBIES,CA_1,CA,d_1,4,2011-01-29,11101,Saturday,Jan,2011,no_event,no_event,no_event,no_event,weekend,0.700195,2.800781
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46881672,FOODS_3_823_WI_3_evaluation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,d_1941,1,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.980469,2.980469
46881673,FOODS_3_824_WI_3_evaluation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,2.480469,0.000000
46881674,FOODS_3_825_WI_3_evaluation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,d_1941,2,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,3.980469,7.960938
46881675,FOODS_3_826_WI_3_evaluation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,d_1941,0,2016-05-22,11617,Sunday,May,2016,no_event,no_event,no_event,no_event,weekend,1.280273,0.000000


In [138]:
sample_submission_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HOBBIES_1_002_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HOBBIES_1_004_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,HOBBIES_1_005_CA_1_validation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60976,FOODS_3_824_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60977,FOODS_3_825_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60978,FOODS_3_826_WI_3_evaluation,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [139]:
df9 = revenue.groupby(["id","d"]).sum()["sale_units"].reset_index()
df9

,id,d,sale_units
0,FOODS_1_001_CA_1_evaluation,d_1,3
1,FOODS_1_001_CA_1_evaluation,d_10,0
2,FOODS_1_001_CA_1_evaluation,d_100,0
3,FOODS_1_001_CA_1_evaluation,d_1000,1
4,FOODS_1_001_CA_1_evaluation,d_1001,0
...,...,...,...
46881672,HOUSEHOLD_2_516_WI_3_evaluation,d_995,0
46881673,HOUSEHOLD_2_516_WI_3_evaluation,d_996,1
46881674,HOUSEHOLD_2_516_WI_3_evaluation,d_997,0
46881675,HOUSEHOLD_2_516_WI_3_evaluation,d_998,0


In [141]:
df9[df9.id == 'FOODS_1_001_CA_1_evaluation']

,id,d,sale_units
0,FOODS_1_001_CA_1_evaluation,d_1,3
1,FOODS_1_001_CA_1_evaluation,d_10,0
2,FOODS_1_001_CA_1_evaluation,d_100,0
3,FOODS_1_001_CA_1_evaluation,d_1000,1
4,FOODS_1_001_CA_1_evaluation,d_1001,0
...,...,...,...
1936,FOODS_1_001_CA_1_evaluation,d_995,0
1937,FOODS_1_001_CA_1_evaluation,d_996,0
1938,FOODS_1_001_CA_1_evaluation,d_997,0
1939,FOODS_1_001_CA_1_evaluation,d_998,1
